In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import json
import os

In [66]:
fanfics = os.listdir('Fanfic_HTML')

remove_fanfics = ['.DS_Store', 'New Beginnings AU.html', 'Book Commentary The Book.html', 'Google Reads Fanfics.html', 'Overheated.html', 'The Internet Is Forever.html', 'Bound.html']
for item in remove_fanfics:
    fanfics.remove(item)

for index, item in enumerate(fanfics):
    fanfics[index] = 'Fanfic_HTML/' + item

In [48]:
def extractStoryRating(classTags):
    #code to extract story rating
    i = 0
    for element in classTags:
        i += 1
        if i == 4:
            story_rating = str(element.find('a').string)
        elif i > 4:
            break
            
    return story_rating

def extractArchiveWarning(classTags):
    i = 0
    for element in classTags:
        i += 1
        if i == 8:
            archiveWarning = str(element.find('a').string)
        elif i > 8:
            break
            
    return archiveWarning

def extractCategory(classTags):
    #code to extract category
    i = 0
    for element in classTags:
        i += 1
        if i == 12:
            story_category = str(element.find('a').string)
        elif i > 12:
            break

    return story_category

def extractFandoms(classTags):
    #code to extract fandoms
    i = 0
    j = 0
    fandom_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Fandom:":
            j = 2 + i
        elif i == j:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += str(obj.string) + ', '
            fandom_text = fandom_text[:-2]

    return fandom_text

def extractRelationships(classTags):
    #code to extract relationships
    i = 0
    j = 0
    story_relationships = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Relationship:":
            j = 2 + i
        elif i == j:
            raw_relationships = element.find_all('a')
            for obj in raw_relationships:
                story_relationships += str(obj.string) + ', '
            story_relationships = story_relationships[:-2]
            
    return story_relationships

def extractCharacters(classTags):
    #code to extract characters
    i = 0
    j = 0
    characters = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Character:":
            j = 2 + i
        elif i == j:
            raw_characters = element.find_all('a')
            for obj in raw_characters:
                characters += str(obj.string) + ', '
            characters = characters[:-2]
            
    return characters

def extractAddTags(classTags):
    #code to extract tags
    i = 0
    j = 0
    additionalTags_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Additional Tags:":
            j = 2 + i
        elif i == j:
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += str(obj.string) + ', '
            additionalTags_text = additionalTags_text[:-2]

    return additionalTags_text

def extractSeries(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Series:":
            j = 2 + i
        elif i == j:
            series = str(element.find('a').string)

    return series

def extractCollections(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    collections = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Collections:":
            j = 2 + i
        elif i == j:
            collections = str(element.find('a').string)

    return collections

def extractPubStats(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Stats:":
            j = 2 + i
        elif i == j:
            pub_stat = str(element.string).strip()
            stat_length = len(pub_stat)
            story_publish_date = pub_stat[11:21]
            story_word_count = pub_stat[stat_length-4:stat_length+1]
            if " " not in story_word_count:
                continue
            else: 
                story_word_count = story_word_count[1:]
    
    return story_publish_date, story_word_count

In [16]:
def extractTags(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    fandom_text = np.nan
    story_character = np.nan
    relationship = np.nan
    addTags = np.nan
    archiveWarning = np.nan
    category = np.nan
    fandoms = np.nan
    pubDate = np.nan
    series = np.nan
    collections = np.nan
    wordCount = np.nan
    tagsClass = soup_obj.find(class_="tags")
    for element in tagsClass:
        if str(element.string) == "Rating:":
            rating = extractStoryRating(tagsClass)
            
        elif str(element.string) == "Archive Warning:":
            archiveWarning = extractArchiveWarning(tagsClass)
            
        elif str(element.string) == "Category:":
            category = extractCategory(tagsClass)
            
        elif str(element.string) == "Fandom:":
            fandom_text = extractFandoms(tagsClass)
        
        elif str(element.string) == "Relationship:":
            relationship = extractRelationships(tagsClass)
            
        elif str(element.string) == "Character:":
            story_character = extractCharacters(tagsClass)
            
        elif str(element.string) == "Additional Tags:":
            addTags = extractAddTags(tagsClass)
            
        elif str(element.string) == "Series:":
            series = extractSeries(tagsClass)
            
        elif str(element.string) == "Collections:":
            collections = extractCollections(tagsClass)
            
        elif str(element.string) == "Stats:":
            pubDate, wordCount = extractPubStats(tagsClass)
            
        i += 1
    
    local_tag_dict = {'rating' : rating,
                'archiveWarnings': archiveWarning,
                'category' : category,
                'fandom' : fandom_text,
                'relationships' : relationship,
                'character': story_character,
                'additionalTags': addTags,  
                'series': series, 
                'collections': collections,
                'pub_date' : pubDate, 
                'word_count' : wordCount
               }
        
    return local_tag_dict

In [24]:
def extractAuthor(soup_obj):
    #code to extract author and fandom info
    for info in soup_obj.title:
        author = str(info).split('-')[1]
        author = author.strip()
    return author

def extractTitle(soup_obj):
    #code to extract title
    i = 0
    for element in soup_obj.find(class_="message"):
        if i == 1:
            title = element.string
        elif i > 1:
            break
        i += 1
        
    return title
    
def extractStory(soup_obj):
    #code to extract title and story text
    i = 0
    story_text = ""
    for element in soup_obj.find(id="chapters"):
        if i == 3:
            if "<span>" in str(soup_obj):
                story_spantag_text = element.find_all('span')
                for obj in story_spantag_text:
                    if obj.string == None:
                        continue
                    else:
                        story_text += str(obj.string) + " "
                        
            elif "<div>" in str(soup_obj):
                story_divtag_text = element.find('div')
                story_ptag_text = story_divtag_text.find_all('p')
                for obj in story_ptag_text:
                    features = obj.contents
                    for feature in features:
                        if "br" in str(feature):
                            continue
                        else:
                            story_text += str(feature) + " " 
                            story_text = story_text.replace("<i>", "")
                            story_text = story_text.replace("</i>", "")
                        
            else:
                story_ptag_text = element.find_all('p')
                if len(story_ptag_text) == 1:
                    story_text = str(element.find('p'))
                    story_text = story_text.replace("<p>", "")
                    story_text = story_text.replace("</p>", "")
                    story_text = story_text.replace("<br/>", "")
                else:
                    for obj in story_ptag_text:
                        if obj.string == None:
                            continue
                        else:
                            story_text += str(obj.string) + " "
                       
        elif i > 3:
            break
            
        i += 1
        
    return story_text


def extractStoryChapters(soup_obj):
    #code to extract title and story text
    i = 0
    j = 0
    story_chapter_text = ""
    chapters = soup_obj.find(id="chapters")
    for element in soup_obj.find(id="chapters"):
        i += 1
        if str(element.string) == "chapter content":
            j = 2 + i
        elif str(element.string) == "/chapter content":
            j = 2 + i
        elif i == j:
            if 'blockquote class="userstuff"' in str(chapters.find(class_="userstuff")):
                k = 0
                m = 0
                for element in chapters:
                    k += 1
                    if str(element.string) == "chapter content":
                        m = 2 + k
                    elif k == m:
                        chapter_ptag_text = element.find_all('p')
                        
            elif "<span>" in str(chapters.find(class_="userstuff")):
                n = 0
                o = 0
                for element in chapters:
                    n += 1
                    if str(element.string) == "chapter content":
                        o = 2 + n
                    elif n == o:
                        chapter_ptag_text = element.find_all('span')
                        
            else:
                userStuff = chapters.find(class_="userstuff")
                chapter_ptag_text = userStuff.find_all('p')
                
            for obj in chapter_ptag_text:
                if obj.string == None:
                    continue
                else:
                    story_chapter_text += str(obj.string) + " "
                    
    return story_chapter_text

def extractStoryURL(soup_obj): 
    #code to extract story URL
    i = 0
    for element in soup_obj.find(class_="message"):
        i += 1
        if i == 7:
            url = element['href']

        else:
            continue
    return url

def createAO3dict(filelist):
    i = 0
    ao3_dict = {}
    for element in filelist:
        soup = BeautifulSoup(open(element), "html.parser")
        url = extractStoryURL(soup)
        tag_dict = extractTags(soup)
        author = extractAuthor(soup)
        title = extractTitle(soup)
        if "chapter content" in str(soup):
            story_text = extractStoryChapters(soup)
        else:
            story_text = extractStory(soup)
        
        story_dict = {'title': title,
                      'author': author,
                      'story_text': story_text, 
                      'url': url,
                     }
        story_dict.update(tag_dict)
        
        ao3_dict[i] = story_dict
        i += 1
    return ao3_dict

In [67]:
ao3_dictionary = createAO3dict(fanfics)

In [68]:
ao3_df = pd.DataFrame.from_dict(ao3_dictionary, orient="index")
ao3_df

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count
0,One Too Many Accidents,kazesuke,"“Sophiiiieee!!!” Loud, wet footsteps slammed a...",http://archiveofourown.org/works/299981,General Audiences,No Archive Warnings Apply,Gen,Howl no Ugoku Shiro | Howl's Moving Castle,NaN,"Howl Pendragon, Sophie, Markl, Calcifer (Howl ...",Genderbending,NaN,Yuletide 2011,2011-12-22,1071
1,The Life Plan,Anonfandom101,Life. ( A brief albeit important prolouge )\n\...,http://archiveofourown.org/works/19198501,Teen And Up Audiences,No Archive Warnings Apply,F/F,Hitchhiker's Guide to the Galaxy - Douglas Ada...,NaN,NaN,NaN,NaN,NaN,2019-06-15,750
2,One Recurse,orphan_account,No one of them could be Cindi Mayweather. It's...,http://archiveofourown.org/works/333825,Mature,No Archive Warnings Apply,F/F,"Metropolis: The Chase Suite - Janelle Monáe, T...",NaN,"Alpha 9000s, Cindi Mayweather","Androids, Clones, Character of Color, Porn Battle",NaN,Porn Battle XIII (Lucky Thirteen),2012-02-06,889
3,"ask, and ye shall receive",Ariaste,"He feels rather daring about it, especially wh...",http://archiveofourown.org/works/19964368,General Audiences,No Archive Warnings Apply,M/M,"Good Omens (TV), Good Omens - Neil Gaiman & Te...",Aziraphale/Crowley (Good Omens),NaN,"a tiny accidental fic, cute and sweet, puns",NaN,My Favorite Ineffable Husband Fics,2019-07-24,441
4,In Gardens,Bitterblue,"Neither of them is sure how to start, but Bit...",http://archiveofourown.org/works/432750,General Audiences,No Archive Warnings Apply,Gen,"Bitterblue - Kristin Cashore, Seven Kingdoms T...",NaN,"Hava (Bitterblue), Bitterblue (Graceling)",NaN,NaN,NaN,2012-06-13,279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,Exchange,swiddershins,"He’d inquired about the coat’s price, carefull...",http://archiveofourown.org/works/14392434,General Audiences,No Archive Warnings Apply,Gen,"Neverwhere - Neil Gaiman, Neverwhere - All Med...",NaN,Marquis de Carabas (Neverwhere),"baby marquis, baby baby baby marquis, Characte...",NaN,NaN,2018-04-22,759
1040,Peeta Mellark: In it to Survive,SilverTonguedWriter,The games were tomorrow. I had no fall back pl...,http://archiveofourown.org/works/1000334,Mature,Choose Not To Use Archive Warnings,M/M,Hunger Games Trilogy - Suzanne Collins,Cato/Peeta Mellark,"Cato (Hunger Games), Peeta Mellark","One Shot, Sex, Blow Jobs",NaN,NaN,2013-10-11,444
1041,Keeping Promises,fourredfruits,Everything happened so fast it seemed like the...,http://archiveofourown.org/works/18372194,General Audiences,Choose Not To Use Archive Warnings,F/M,"The Umbrella Academy (TV), The Umbrella Academ...","Number Five | The Boy & Vanya Hargreeves, Numb...","Number Five | The Boy (Umbrella Academy), Vany...",NaN,NaN,NaN,2019-04-06,808
1042,Interlude: It grows back,kaleigh,,http://archiveofourown.org/works/21219572,Not Rated,No Archive Warnings Apply,F/F,"Addams Family - All Media Types, Addams Family...",Wednesday Addams/Original Female Character(s),"Wednesday Addams, Pubert Addams, Lurch (Addams...","Haircuts, Sibling Bonding",Love is a four letter word,NaN,2019-10-29,959


In [69]:
ao3_df.word_count.astype('int32')

0       1071
1        750
2        889
3        441
4        279
        ... 
1039     759
1040     444
1041     808
1042     959
1043     100
Name: word_count, Length: 1044, dtype: int32

In [70]:
#drop all rows where story_text is a blank string, for now
blankStory_df = ao3_df[ao3_df["story_text"] == '']

In [71]:
ao3_df = ao3_df.drop(blankStory_df.index)
ao3_df[ao3_df["story_text"] == '']

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count


In [65]:
ao3_df.to_csv("ao3_data.csv", index=False)

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count
650,Overheated,Su_Whisterfield,,http://archiveofourown.org/works/30107565,Explicit,No Archive Warnings Apply,M/M,X-Men (Comicverse),Logan/Kurt Wagner,"Kurt Wagner, Logan (X-Men)","sauna sex, Tail Sex",NaN,NaN,2021-03-17,: 0
768,The Internet Is Forever,Miffy,,http://archiveofourown.org/works/7310356,General Audiences,Choose Not To Use Archive Warnings,F/M,"Hunger Games Trilogy - Suzanne Collins, Hunger...","Katniss Everdeen/Peeta Mellark, Annie Cresta/F...","Katniss Everdeen, Peeta Mellark, Primrose Ever...","Social Media, Fluff, Alternate Universe - Mode...",Blue Skies Are Coming,NaN,2016-06-26,: 0
830,Bound,Rav3nB1ack,"\n<iframe allowfullscreen="""" frameborder=""0"" h...",http://archiveofourown.org/works/23647354,General Audiences,Choose Not To Use Archive Warnings,F/M,"Pacific Rim (Movies), 47 Ronin (2013), King Ar...","Raleigh Becket/Mako Mori, Raleigh Becket & Mak...","Mako Mori, Raleigh Becket, Arthur (King Arthur...","Fanvids, Alternate Universe, Alternate Univers...",NaN,NaN,2020-04-14,: 0
